# Data preprocessing

In this notebook we extract the data from the SQL database provided by ProPublica. We do this (rather than just use their compas-scores.csv) for a number of reasons: 
* The compas-scores.csv only contains the decile scores, not the raw scores. We want to have access to both. 
* The compas-scores.csv does not contain the recommended supervision level scores. 
* The compas-scores.csv does not contain marital status information. 
* We want to add an 'age-at-first-arrest' feature, as the compas practisioner's guide lists this as one of the inputs to the compas program. 
* For the third data analysis we want to extract all the scores for the three types of compas risk assessments as separate files.

To do this, we import the SQL tables from `compas.db` to pandas dataframes. For the first four points above, we compute the desired information and merge the tables into a single dataframe which is then written to `data/compas.csv`. This will be csv used for our first and second data analyses of the paper. Then, for the last point above, we process just the `compas` table into three further dataframes, containing the raw, decile, and text scores of all the rows corresponding to the three types of compas risk scores. These are then also written to csv files, for our third data analysis of the paper. 

### Imports

In [1]:
import pandas as pd
import numpy as np
import operator
import random
import sqlite3
from z3 import * # must be at least version 4.12.2. 
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from time import time
from tqdm import tqdm
from tabulate import tabulate
from itertools import product
from datetime import datetime

pd.set_option('future.no_silent_downcasting', True)

# Class for a "functional" dictionary, which can be also be accessed with round brackets. 
class fdict(dict):
    def __call__(self, k):
        return self[k] 

### Extract frames from the SQL database file

In [2]:
# Replace 'your_database.db' with the path to your .db file
database_path = 'data/compas.db'

# Establish a connection to the database
conn = sqlite3.connect(database_path)

# Create a cursor object
cursor = conn.cursor()

# Execute a query to retrieve the names of all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all table names except the 'summary' table, which is empty. 
tables = [table[0] for table in cursor.fetchall() if table[0] != "summary"]

# Read the tables as dataframes and store in a dict. 
dfd = {table : pd.read_sql_query(f"SELECT * FROM {table}", conn) for table in tables}

# Close the connection
conn.close()

### Compute age-at-first-arrest column

In [3]:
# Group the dfd['casearrest'] by person_id and keep only the row with the earliest 'arrest_date' date. 
first_arrest = dfd['casearrest'].sort_values('arrest_date').groupby('person_id').first().reset_index() 
first_arrest = first_arrest[['arrest_date', 'person_id']]

# Copy the people df - this will be appended with information from other tables. 
ext_people = dfd['people'].copy()

# Remove the trailing "00:00:00.000000" from dob. 
ext_people['dob'] = ext_people['dob'].apply(lambda x: x.split(' ')[0])

# Rename the 'id' column in ext_people to 'person_id', for merging with other tables.
ext_people = ext_people.rename(columns={'id': 'person_id'})

# Merge ext_people with first_arrest.
ext_people = ext_people.merge(right=first_arrest, on='person_id', how='left', suffixes=(None, '_fa'))

# Replace NaN values in the 'arrest_date' column with "2016-01-01 00:00:00.000000".
ext_people['arrest_date'] = ext_people['arrest_date'].fillna("No_arrest")

# Change the column type of arrest_date to string, and remove trailing "00:00:00.000000".
ext_people['arrest_date'] = ext_people['arrest_date'].astype(str)
ext_people['arrest_date'] = ext_people['arrest_date'].apply(lambda x: x.split(' ')[0])

# Drop extraneous columns. 
ext_people = ext_people.drop(columns=['first', 'last', 'race', 'name', 'compas_screening_date', 'score_text', 'violent_recid', 'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number', 'c_days_from_compas', 'c_arrest_date', 'c_offense_date', 'c_charge_degree', 'c_charge_desc', 'is_recid', 'num_r_cases', 'r_case_number', 'r_charge_degree', 'r_days_from_arrest', 'r_offense_date', 'r_charge_desc', 'r_jail_in', 'r_jail_out', 'is_violent_recid', 'num_vr_cases', 'vr_case_number', 'vr_charge_degree', 'vr_offense_date', 'vr_charge_desc', 'age_cat'])

# Compute difference in dates (in string format) in years. 
def time_diff(s1, s2, s3):
    if s2 == "No_arrest":
        return s3
    else:
        format = "%Y-%m-%d"
        d1 = datetime.strptime(s1, format)
        d2 = datetime.strptime(s2, format)
        return round((d2 - d1).days / 365.25)

# Compute the time difference in years between date of birth (dob) and first arrest date (arrest_date). 
ext_people['age_at_first_arrest'] = ext_people.apply(lambda row: time_diff(row['dob'], row['arrest_date'], row['age']), axis=1)

# Drop the arrest_date and dob columns.
ext_people = ext_people.drop(columns=['arrest_date', 'dob'])

### Join marital status and risk scores. 

In [4]:
compas_df = dfd['compas'].copy()

# Filter rows with invalid scores. 
compas_df = compas_df[compas_df["score_text"] != "N/A"]

# Remove COPMAS assessments from the df other than the first. 
# First, sort the DataFrame by person_id and compas_assessment_id
compas_df_sorted = compas_df.sort_values(by=['person_id', 'compas_assessment_id'])
first_screening = compas_df_sorted.groupby('person_id').first().reset_index()
first_screening_df = compas_df_sorted.merge(first_screening[['person_id', 'compas_assessment_id']], on=['person_id', 'compas_assessment_id'])

# Drop extraneous columns. 
cols = ['id', 'type_of_assessment', 'raw_score', 'decile_score', 'score_text', 'person_id', 'marital_status', 'rec_supervision_level']
dcols = [col for col in first_screening_df.columns if col not in cols]
first_screening_df = first_screening_df.drop(columns=dcols)

# Each person now has three rows in the column corresponding to the recidivism, 
# violent recidivism, and FTA risk scores. We now change these into three separate 
# columns rather than three rows, using pivot. 
pivoted_df = first_screening_df.pivot(index='person_id', columns='type_of_assessment', values=['raw_score', 'decile_score', 'score_text'])
pivoted_df.columns = ['_'.join(col).strip().replace(' ', '_').lower() for col in pivoted_df.columns.values]
pivoted_df.reset_index(inplace=True)

# Merge the pivoted DataFrame with the original DataFrame to retain other columns
# Drop duplicates to ensure each person_id has only one row
other_columns = first_screening_df.drop(['raw_score', 'decile_score', 'score_text', 'type_of_assessment'], axis=1).drop_duplicates(subset='person_id')
final_df = pd.merge(other_columns, pivoted_df, on='person_id')
ext_people = ext_people.merge(final_df, on='person_id', how='left')

# Drop some unneeded columns.
ext_people = ext_people.drop(columns=['id', 'person_id', 'decile_score'])

### Rename some columns and values

In [5]:
# Shorten the column names. 
repl = {
    'raw_score' : 'rs',
    'decile_score' : 'ds',
    'rec_supervision_level' : 'slevel',
    'risk_of_failure_to_appear' : 'fta', 
    'risk_of_recidivism' : 'grecid', 
    'risk_of_violence' : 'vrecid',
    'score_text_fta' : 'fta_text',
    'score_text_grecid' : 'grecid_text', 
    'score_text_vrecid' : 'vrecid_text',
    'sex' : 'male',
}
for k in repl:
    ext_people.columns = ext_people.columns.str.replace(k, repl[k])

# Change the type of the 'ds_*' columnns to int.
for c in ext_people.columns:
    if 'ds_' in c:
        ext_people[c] = ext_people[c].astype(float).astype(pd.Int64Dtype())

# Change 'male' column values to 1 and 0. 
r_dict = {"Male" : 1, "Female" : 0}
ext_people["male"] = ext_people["male"].replace(r_dict)

# Change sex column datatype to int64.
ext_people["male"] = ext_people["male"].astype("int64")

### Make the lists of binnings.

In [6]:
df = dfd["compas"].copy()
df = df[df["score_text"] != "N/A"]

fta_bins = df[df["type_of_assessment"] == "Risk of Failure to Appear"]
fta_bins = fta_bins[["raw_score", "decile_score", "score_text"]].dropna()

grecid_bins = df[df["type_of_assessment"] == "Risk of Recidivism"]
grecid_bins = grecid_bins[["raw_score", "decile_score", "score_text"]].dropna()

vrecid_bins = df[df["type_of_assessment"] == "Risk of Violence"]
vrecid_bins = vrecid_bins[["raw_score", "decile_score", "score_text"]].dropna()


### Write out to `.csv` files

In [7]:
# Write the ext_people df to a csv file in the 'data' subfolder. 
ext_people = ext_people.dropna()
ext_people.to_csv('data/compas.csv', index=False)

# Write out the bin frames.
fta_bins.to_csv('data/fta_bins.csv', index=False)
grecid_bins.to_csv('data/grecid_bins.csv', index=False)
vrecid_bins.to_csv('data/vrecid_bins.csv', index=False)

print("Finished writing frames to csv files.")

Finished writing frames to csv files.
